*Идея такая*
* выборку разбили на train и val
* берем 3 предобученных нейронки: ResNet18, ResNet34, ResNet50
* запускаем цикл (в моем случае 250 итераций), где на каждом шаге выбираем случайным образом одну из трех нейросетей, случайно генерируем из заданного диапазона кол-во эпох, оптимизатор (SGD или Adam), шаг обучения, momentum, scheduler и т.д. А также параметры аугментаций
* Обучаем и смотрим кач-во на валидации. Если оно не меньше 0.75, то оставляем эту сеть.
* После цикла берем результаты "лучших" нейронок и усредняем их ответ
* Получилось 90%

In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import matplotlib.pyplot as plt
import os
import shutil 
import zipfile
from torchvision import transforms, models
from tqdm import tqdm
from IPython.display import FileLink

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# распаковываем архив
with zipfile.ZipFile('../input/platesv2/plates.zip','r') as z:
    z.extractall('.')
    
print(os.listdir('./plates/'))

data_root = './plates'
print(os.listdir(data_root))

In [ ]:
train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

os.remove(os.path.join(data_root, 'train/cleaned', '.DS_Store'))
os.remove(os.path.join(data_root, 'train/dirty', '.DS_Store'))

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))
        

os.listdir('./val/dirty')

In [ ]:
def show_input(input_tensor, title=''):
    std = [0.229, 0.224, 0.225]
    mean = [0.485, 0.456, 0.406]
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image)
    plt.title(title)
    plt.show()
    plt.pause(0.001)

In [ ]:
def get_random_train_transforms():
    '''
    Функция для случайной генерации параметров аугментации
    '''
    min_scale = np.random.random() / 1.43 + 0.3  # 0.3 ... 1
    
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(min_scale, 1.0)),     
        transforms.RandomHorizontalFlip(p=np.random.random()),
        transforms.RandomVerticalFlip(p=np.random.random()),
        transforms.RandomRotation(degrees=90*np.random.random()),
        transforms.ColorJitter(brightness=0.7*np.random.random()),
        transforms.ColorJitter(contrast=0.7*np.random.random()),
        transforms.ColorJitter(saturation=0.7*np.random.random()),
        transforms.ColorJitter(hue=0.3*np.random.random()),
        transforms.RandomGrayscale(p=0.4*np.random.random()),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    
    return train_transforms

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    
    val_loss = 0.
    val_acc = 0.
    train_loss = 0.
    train_acc = 0.
      
    for epoch in tqdm(range(num_epochs)):

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.
    
            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()
                        
                if epoch == num_epochs - 1:
                    # statistics
                    running_loss += loss_value.item()
                    running_acc += (preds_class == labels.data).float().mean().item()
                        
            if epoch == num_epochs - 1:
                if phase == 'val':                  
                    val_loss = running_loss / len(dataloader)
                    val_acc = running_acc / len(dataloader)
                else:
                    train_loss = running_loss / len(dataloader)
                    train_acc = running_acc / len(dataloader)
        
    return model, val_loss, val_acc, train_loss, train_acc

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pretrained_models = [
    models.resnet18(pretrained=True), 
    models.resnet34(pretrained=True), 
    models.resnet50(pretrained=True)
]

# Disable grad for all conv layers
for model in pretrained_models:
    for param in model.parameters():
        param.requires_grad = False
    model.fc = torch.nn.Linear(model.fc.in_features, 2)    
    model = model.to(device)
    
loss = torch.nn.CrossEntropyLoss()

In [ ]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
batch_size = 8

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
num_successes = 0

for num_nn in range(250):
    train_transforms = get_random_train_transforms()
    train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
    
    # выбираем модель
    num_model = np.random.randint(0, 3)
    model = pretrained_models[num_model]
    model.fc = torch.nn.Linear(model.fc.in_features, 2)    
    model.to(device)
    
    # оптимизатор
    num_optimizer = np.random.randint(0, 2)
    lr = 1e-3 * np.random.randint(1, 11)
    if num_optimizer == 0:
        momentum = 1e-2 * np.random.randint(70, 96)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    else:
        momentum = 0
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # scheduler
    step_size = np.random.randint(7, 26)
    gamma = 0.1 * np.random.randint(1, 6)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    # кол-во эпох
    num_epochs = np.random.randint(30, 201)
    
    print("NN № {0}, num_epochs={1}".format(num_nn+1, num_epochs), flush=True)
    
    model, val_loss, val_acc, train_loss, train_acc = train_model(model, loss, optimizer, scheduler, num_epochs=num_epochs);
    
    status = "success" if val_acc >= 0.75 else "fail"
    print("loss = {0}, acc = {1}, status: {2}".format(val_loss, val_acc, status), flush=True)
    
    if status == "success":
        
        num_successes += 1
        
        model.eval()

        test_predictions = []
        test_img_paths = []
        for inputs, labels, paths in tqdm(test_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                preds = model(inputs)
            test_predictions.append(
                torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
            test_img_paths.extend(paths)

        test_predictions = np.concatenate(test_predictions)
        
        
        data_for_analysis = [train_acc, train_loss, val_acc, val_loss, 
                             num_model, num_optimizer, lr, momentum, step_size, gamma, num_epochs,
                             *train_transforms.transforms[:9]]
        labels_for_analysis = ["train_acc", "train_loss", "val_acc", "val_loss", 
                               "num_model", "num_optimizer", "lr", "momentum", "step_size", "gamma", "num_epochs",
                               "transform_1", "transform_2", "transform_3", "transform_4", "transform_5",
                               "transform_6", "transform_7", "transform_8", "transform_9"]
        
        if num_successes == 1:
            submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'pred_1': test_predictions})
            submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
            submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
            submission_df.set_index('id', inplace=True)
            analysis_df = pd.DataFrame.from_dict({'params': labels_for_analysis, 'pred_1': data_for_analysis})
            analysis_df.set_index('params', inplace=True)
        else:
            submission_df['pred_' + str(num_successes)] = test_predictions
            analysis_df['pred_' + str(num_successes)] = data_for_analysis
    

submission_df.head()

In [ ]:
file_name = 'submission.csv'
submission_df.to_csv(file_name)

FileLink(file_name)

In [ ]:
file_name = 'analysis.csv'
analysis_df.to_csv(file_name)

FileLink(file_name)

In [ ]:
col = [str(col_name) for col_name in submission_df.columns if 'pred' in col_name]
submission_df['label'] = submission_df[col].mean(axis=1)
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.76 else 'cleaned')
submission_df.drop(col, axis=1, inplace=True)

file_name = 'submission.csv'
submission_df.to_csv(file_name)

FileLink(file_name)

In [ ]:
!rm -rf train val test